In [1]:
# needed liberaies

import pandas as pd
from sklearn import preprocessing
import random
import numpy as na
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Import the data and print it

data =pd.read_csv('data.csv',sep=';')
names = ['x','y']

In [3]:
#scale the data
scaler = preprocessing.MinMaxScaler()

scaled_df=scaler.fit_transform(data)
scaled_df = pd.DataFrame(scaled_df, columns=names)

# scaled data
scaled_df = scaled_df.round(2)

In [4]:
# generate two random weights and print their value
a =random.uniform(0, 1)
b =random.uniform(0, 1)
a=.45
b=.75
r =0.01
print('Intial weights are {} and {}and learning rate is {}'.format(a,b,r))

Intial weights are 0.45 and 0.75and learning rate is 0.01


In [5]:
# generate empty column for predict value of y(Yp), difference between(y-Yp) and SSE
def prepare_data(data,a,b,r):
    
    # create all the columns needed
    data['Yp']=data.apply(lambda _: '', axis=1)
    data['y-Yp']=data.apply(lambda _: '', axis=1)
    data['SSE']=data.apply(lambda _: '', axis=1)
    data['derA_SSE']=data.apply(lambda _: '', axis=1)
    data['derB_SSE']=data.apply(lambda _: '', axis=1)
    
    for i in range(len(data)):
        data['Yp'].iloc[i] = (a + (b * data['y'].iloc[i]))
        data['y-Yp'].iloc[i] = data['y'].iloc[i] - data['Yp'].iloc[i]
        data['SSE'].iloc[i]= (na.power(data['y-Yp'].iloc[i],2) * .5)
        data['derA_SSE'].iloc[i]= data['y-Yp'].iloc[i] * -1
        data['derB_SSE'].iloc[i]= data['y-Yp'].iloc[i] * data['x'].iloc[i] * -1
    return data

In [6]:
# Fubction to calculate SSE, dSSE/da, dSSE/db
def SSE(data):
    list =[]
    SSEDATA = pd.DataFrame(columns=['SSE','derA_SSE','derB_SSE'])
    list.append(sum(data['SSE']))
    list.append(sum(data['derA_SSE']))
    list.append(sum(data['derA_SSE']))
    SSEDATA.loc[0] = list
    return (SSEDATA)

In [7]:
def grad_descent_algo(data,a,b,r,total_iter):
    final =[]
    for i in range(total_iter):
        
        if i == 0:
           
            SSE_first=SSE(prepare_data(data,a,b,r))
            final.append(SSE_first['SSE'][0])
        if i == 1:
            # Update the vallues of a and b
            #print(SSE_first['derA_SSE'])
            #print(SSE_first['derB_SSE'])
            a = a-(r * SSE_first['derA_SSE'])[0]
            b = b-(r * SSE_first['derB_SSE'])[0]

            SSE_second = SSE(prepare_data(data,a,b,r))
            final.append(SSE_second['SSE'][0])
        if i == 2:
            
            if (SSE_first['SSE'][0] > SSE_second['SSE'][0]):
                a = a-(r * SSE_second['derA_SSE'])[0]
                b = b-(r * SSE_second['derB_SSE'])[0]
            else:
                a = a + (r * SSE_second['derA_SSE'])[0]
                b = b + (r * SSE_second['derB_SSE'])[0]
        if i>=2:
            temp_prev = SSE_second['SSE'][0]
            temp_next= SSE(prepare_data(data,a,b,r))
            final.append(temp_next['SSE'][0])
            if (temp_next['SSE'][0] > temp_prev):
                a = a-(r * temp_next['derA_SSE'])[0]
                b = b-(r * temp_next['derB_SSE'])[0]
            else:
                a = a+(r * temp_next['derA_SSE'])[0]
                b = b+(r * temp_next['derB_SSE'])[0]
            temp_prev = temp_next['SSE'][0]
    
    return(final,a,b)
                   

In [10]:
def plot(df):
    # Standard plotly imports
    import plotly.plotly as py
    import plotly.graph_objs as go
    from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
    import cufflinks
    cufflinks.go_offline(connected=True)
    init_notebook_mode(connected=True)

    return(pd.DataFrame(df).iplot(
    mode='lines+markers',
    y=0,
    opacity=0.8,
    xTitle='No of Iterations',
    yTitle='Sum of Square errors',
    title='Gradient descent Algorithm : value of SSE'))

In [12]:
def main(iter):
    values = grad_descent_algo(scaled_df,a,b,r,iter)
    plot(values[0])
    print('Value of SSE is {}'.format(values[0][-1]))
    print('Value of a is {}'.format(values[1]))
    print('Value of b is {}'.format(values[2]))
    print('Lowest value of SSE is {}'.format(min(values[0])))
if __name__== "__main__":
    main(150)

Value of SSE is 0.3492034705507686
Value of a is 0.41854223460018725
Value of b is 0.7185422346001872
Lowest value of SSE is 0.3308588152113085
